In [1]:
%load_ext autoreload
%autoreload

In [2]:
from pathlib import Path
home_path = '/home/eromanenkova/'

In [3]:
import sys
sys.path.append('/home/eromanenkova/')
sys.path.append('/home/eromanenkova/mds20_replearning')

In [4]:
from mds20_replearning.scripts.skip_thoughts.pl_model import SkipThoughtsModule
from mds20_replearning.data.language.load import load_data

In [5]:
model = SkipThoughtsModule(vocab_size=20000, embedding_dim=620, encoder_dim=2400, lr=2e-4) 

In [6]:
model = model.load_from_checkpoint(checkpoint_path="2epochs_stv_loss_2_276.ckpt")

In [7]:
from mds20_replearning.scripts.pl_data import DefaultDataModule

In [8]:
import pickle
with open('/home/eromanenkova/mds20_replearning/notebooks/w2i_mapping.pickle', 'rb') as f:
    predefined_vocab_mapping = pickle.load(f)

In [9]:
from functools import partial
load_func = partial(load_data, name='review_polarity')
data_module = DefaultDataModule(batch_size=128, load_func=load_func, data_path='/home/eromanenkova/mds20_replearning/data/txt_sentoken',
                               url='http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz', 
                               download_root = '/home/eromanenkova/mds20_replearning/data/',
                               filename = 'review_polarity', 
                               w2i_mapping=predefined_vocab_mapping, shuffle=True, valid_split=0.4)

In [10]:
from mds20_replearning.classification.model import ClassificationModule

In [11]:
from torch.nn import Sequential
stv_encoder = Sequential(model.skipthoughts.embedding, model.skipthoughts.encoder)

In [12]:
stv_encoder.encoder_dim = model.skipthoughts.encoder_dim

In [13]:
linear_model = ClassificationModule(stv_encoder, freeze=True, lr=2e-4, batch_first=False)

In [14]:
import pytorch_lightning as pl
trainer = pl.Trainer(gpus=[1], max_epochs=90)  # set required resources here

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


In [15]:
trainer.fit(linear_model, datamodule=data_module)

/home/eromanenkova/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/eromanenkova/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/eromanenkova/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/eromanenkova/anaconda3/lib/python3.7/site-packages/tensorf

/home/eromanenkova/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/eromanenkova/anaconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

Traceback (most recent call last):
  File "/home/eromanenkova/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/eromanenkova/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/eromanenkova/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/eromanenkova/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/eromanenkova/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/home/eromanenkova/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/eromanenkova/anaconda3/lib/python3.7/multiprocessing/connectio